# Notebook to generate psychometrics plot given sessions & conditions

In [32]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

import datajoint as dj
from datetime import date, timedelta
import pandas as pd
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches

from matplotlib import cm
from u19_pipeline import utility
from inspect import getmembers, isfunction


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


### Datajoint configuration and Connection to DB

In [57]:
utility.basic_dj_configuration(dj)
dj.conn()
acquisition = dj.create_virtual_module('acquisition', 'u19_acquisition')
behavior = dj.create_virtual_module('behavior', 'u19_behavior')

## Filter params

In [119]:
#Filter Dates
filter_dates = True
num_days_check = 60
current_date = date.today()
first_date_to_check = current_date - timedelta(days=num_days_check)
dates        = [first_date_to_check.strftime("%Y-%m-%d"), current_date.strftime("%Y-%m-%d")]
date_label   = ' and '.join(f'"{w}"' for w in dates)
date_query   = 'session_date between ' + date_label 


## Filter rigs
filter_rigs = False
rigs = ['165I-Rig1-T',
        '165I-Rig2-T',
        '165I-Rig3-T',
        '165I-Rig4-T',
        '165A-Rig5-T',
        '165A-Rig6-T',
        '165A-Rig7-T',
        '165A-Rig8-T', 
        '170b-Imaging0642',
        '185f-rig1',
        '188-Rig1',
        '188-Rig2',
        'BezosMeso',
        'C42-Bay2-Rig1-I']
rigs_label = ', '.join(f'"{w}"' for w in rigs)
rigs_query = 'session_location in (' + rigs_label + ')'


#Filter users
filter_user = True
users = ['efonseca_ef255_actgp002']
users = ['subject_fullname like "' + x + '%"' for x in users]
user_query = ' or '.join(f'{w}' for w in users) 

#Filter extras
not_testuser     = 'subject_fullname not like "testuser%"'
not_testsubjects = 'subject_fullname not like "%_test"'
not_bad_sessions = 'is_bad_session = 0'
extra_queries    = [not_testuser, not_testsubjects, not_bad_sessions]

columns_session_table = ['subject_fullname', 'session_date', 'session_number', 'session_location', 'session_performance', 'task', 'level', 'stimulus_bank', 'session_protocol', 'num_trials']
columns_trial_table   = ['subject_fullname', 'session_date', 'session_number', 'block', 'trial_idx', 'trial_type', 'choice']


### Create query based on filter params

In [120]:
query = []
if filter_rigs:
    query.append(rigs_query)
    
if filter_dates:
    query.append(date_query)
    
if filter_user:
    query.append(user_query)

#All extra stuff to filter
for i in extra_queries:
    query.append(i)
    
# Filter by all conditions selected
session_table_filtered = acquisition.Session
for filter_key in query:
    session_table_filtered = session_table_filtered & filter_key
    
session_df = pd.DataFrame(session_table_filtered.fetch(*columns_session_table, as_dict=True))
session_df

,subject_fullname,session_date,session_number,session_location,task,level,stimulus_bank,session_performance,num_trials,session_protocol
0,efonseca_ef255_actgp002,2022-06-06,0,165A-Rig8-T,Towers,2,-- NO STIM BANK MODE --,60.5381,223,LSTT_TrialStructure_GradualParamChange_EF.m LS...
1,efonseca_ef255_actgp002,2022-06-07,0,165A-Rig8-T,Towers,2,-- NO STIM BANK MODE --,48.0000,100,LSTT_TrialStructure_GradualParamChange_EF.m LS...
2,efonseca_ef255_actgp002,2022-06-09,0,165A-Rig8-T,Towers,2,-- NO STIM BANK MODE --,82.0988,324,LSTT_TrialStructure_GradualParamChange_EF.m LS...
3,efonseca_ef255_actgp002,2022-06-10,0,165A-Rig8-T,Towers,3,-- NO STIM BANK MODE --,51.3514,222,LSTT_TrialStructure_GradualParamChange_EF.m LS...
4,efonseca_ef255_actgp002,2022-06-11,0,165A-Rig8-T,Towers,3,-- NO STIM BANK MODE --,59.2105,228,LSTT_TrialStructure_GradualParamChange_EF.m LS...
5,efonseca_ef255_actgp002,2022-06-13,0,165A-Rig8-T,Towers,3,-- NO STIM BANK MODE --,62.5000,304,LSTT_TrialStructure_GradualParamChange_EF.m LS...
6,efonseca_ef255_actgp002,2022-06-14,0,165A-Rig8-T,Towers,3,-- NO STIM BANK MODE --,59.7270,293,LSTT_TrialStructure_GradualParamChange_EF.m LS...
7,efonseca_ef255_actgp002,2022-06-15,0,165A-Rig8-T,Towers,3,-- NO STIM BANK MODE --,60.3015,199,LSTT_TrialStructure_GradualParamChange_EF.m LS...
8,efonseca_ef255_actgp002,2022-06-16,0,165A-Rig8-T,Towers,3,-- NO STIM BANK MODE --,64.1350,237,LSTT_TrialStructure_GradualParamChange_EF.m LS...
9,efonseca_ef255_actgp002,2022-06-17,0,165A-Rig8-T,Towers,3,-- NO STIM BANK MODE --,65.1282,390,LSTT_TrialStructure_GradualParamChange_EF.m LS...


## Protocols used

In [121]:

protocol_df = session_df.groupby('session_protocol').agg({'subject_fullname': [('num_sessions', 'count'), ('num_subjects', 'nunique')]})
protocol_df.columns = protocol_df.columns.droplevel()
#protocol_df = protocol_df.reset_index()
protocol_df = protocol_df.sort_values(by='num_sessions', ascending=False)
protocol_df

,num_sessions,num_subjects
session_protocol,,
LSTT_TrialStructure_GradualParamChange_EF.m LSTT_TrialStructure_GradualParamChange_World_EF.mat LSTT_Levels_GradualParamChange_EF2,23,1


## "Basic mean, std & zscore for num_trials"

In [122]:
session_p_df = session_df.copy()

#Sort by subject, date and session number
session_p_df = session_p_df.sort_values(by=['subject_fullname', 'session_date', 'session_number'], ascending=[True, False, False])
session_p_df = session_p_df.reset_index(drop=True)

# Group metrics for each subject (total num sessions, mean num_trials, std num_trials)
num_sessions_df = session_p_df.groupby('subject_fullname').agg({'session_date': [('total_sessions', 'count')], \
     'num_trials': [('avg_trials', 'mean'), ('std_trials', 'std')]})
num_sessions_df.columns = num_sessions_df.columns.droplevel()
num_sessions_df = num_sessions_df.reset_index()

# Merge session count to session df
session_p_df = session_p_df.merge(num_sessions_df)

session_p_df['z_score_trials'] = (session_p_df['num_trials'] - session_p_df['avg_trials']) / session_p_df['std_trials']

# Keep only today's session
session_p_df = session_p_df.loc[session_p_df['session_date'] == current_date,:]
session_p_df = session_p_df.reset_index(drop=True)
session_p_df

,subject_fullname,session_date,session_number,session_location,task,level,stimulus_bank,session_performance,num_trials,session_protocol,total_sessions,avg_trials,std_trials,z_score_trials
0,efonseca_ef255_actgp002,2022-07-05,0,165A-Rig8-T,Towers,2,-- NO STIM BANK MODE --,74.0741,54,LSTT_TrialStructure_GradualParamChange_EF.m LS...,23,287.478261,102.912527,-2.268706


## Get trial by trial data

In [125]:
# Filter by all conditions selected
trial_table_filtered = acquisition.Session * behavior.TowersBlock.Trial()
for filter_key in query:
    trial_table_filtered = trial_table_filtered & filter_key

trials_df = pd.DataFrame(trial_table_filtered.fetch(*columns_trial_table, as_dict=True))

trials_df

,subject_fullname,session_date,session_number,block,trial_idx,trial_type,choice
0,efonseca_ef255_actgp002,2022-06-06,0,1,1,R,L
1,efonseca_ef255_actgp002,2022-06-06,0,1,2,R,R
2,efonseca_ef255_actgp002,2022-06-06,0,1,3,L,R
3,efonseca_ef255_actgp002,2022-06-06,0,1,4,L,L
4,efonseca_ef255_actgp002,2022-06-06,0,1,5,L,L
...,...,...,...,...,...,...,...
6553,efonseca_ef255_actgp002,2022-07-01,0,11,51,R,R
6554,efonseca_ef255_actgp002,2022-07-01,0,11,52,R,L
6555,efonseca_ef255_actgp002,2022-07-01,0,11,53,R,L
6556,efonseca_ef255_actgp002,2022-07-01,0,11,54,R,R


In [126]:
sort_columns = ['subject_fullname', 'session_date', 'session_number', 'block', 'trial_idx']
session_columns = ['subject_fullname', 'session_date', 'session_number']
trials_df = trials_df.sort_values(by=sort_columns, ascending=[True, False, False, True, True])
trials_df = trials_df.reset_index(drop=True)


# Correct trials, correct to left and correct to right
trials_df['left_trial']    = (trials_df['trial_type'] == 'L').astype(int)
trials_df['right_trial']    = (trials_df['trial_type'] == 'R').astype(int)

trials_df['cum_left_trials'] = trials_df.groupby(['subject_fullname', 'session_date', 'session_number'])['left_trial'].cumsum()
trials_df['cum_right_trials'] = trials_df.groupby(['subject_fullname', 'session_date', 'session_number'])['right_trial'].cumsum()

trials_df['correct_trial'] = (trials_df['trial_type'] == trials_df['choice']).astype(int)
trials_df['correct_left']  = ((trials_df['correct_trial'] == 1) & (trials_df['trial_type'] == 'L')).astype(int)
trials_df['correct_right'] = ((trials_df['correct_trial'] == 1) & (trials_df['trial_type'] == 'R')).astype(int)

trials_df['cum_correct_trials'] = trials_df.groupby(['subject_fullname', 'session_date', 'session_number'])['correct_trial'].cumsum()
trials_df['cum_correct_left_trials'] = trials_df.groupby(['subject_fullname', 'session_date', 'session_number'])['correct_left'].cumsum()
trials_df['cum_correct_right_trials'] = trials_df.groupby(['subject_fullname', 'session_date', 'session_number'])['correct_right'].cumsum()

trials_df = trials_df.loc[~trials_df.duplicated(subset=session_columns, keep='last'), :]
trials_df = trials_df.reset_index(drop=True)

trials_df['bias'] = (trials_df['cum_correct_right_trials'] / trials_df['cum_right_trials']) - (trials_df['cum_correct_left_trials'] / trials_df['cum_left_trials'])

bias_df = trials_df[session_columns + ['bias']]

bias_df

,subject_fullname,session_date,session_number,bias
0,efonseca_ef255_actgp002,2022-07-01,0,0.028315
1,efonseca_ef255_actgp002,2022-06-30,0,-0.023842
2,efonseca_ef255_actgp002,2022-06-29,0,0.015578
3,efonseca_ef255_actgp002,2022-06-28,0,-0.014804
4,efonseca_ef255_actgp002,2022-06-27,0,-0.041739
5,efonseca_ef255_actgp002,2022-06-25,0,-0.019077
6,efonseca_ef255_actgp002,2022-06-24,0,-0.157012
7,efonseca_ef255_actgp002,2022-06-23,0,-0.074074
8,efonseca_ef255_actgp002,2022-06-22,0,-0.124675
9,efonseca_ef255_actgp002,2022-06-21,0,-0.172401


In [98]:
pd.options.display.max_rows

200

In [107]:
pd.options.display.max_rows = 60